Use _ to indicate these variables are private to the class.  
To remind myself not to change them directly in code outside the class.  
Instead, use the class's methods, which check for consequences (validation etc) of a variable change.  

In [34]:
from random import randint

# Dictionary to map Card face to Numerical value
cards_dict = {'2':2,
          '3':3,
          '4':4,
          '5':5,
          '6':6,
          '7':7,
          '8':8,
          '9':9,
          '10':10,
          'J':10,
          'Q':10,
          'K':10,
          'A':11} # We'll handle soft Aces in code.

class Player:
    
    # initialize the attributes
    def __init__(self, name):
        self.name = name 
        self.hand = []
        self.status = 'open' # 'open', 'win', 'stand' (includes push), lose' (bust, dealer natural blackjack)	
    
    def print_hand(self):
        hand_formatted = ' '.join(self.hand)
        print('{}: {}'.format(self.name, hand_formatted))
        
    def deal(self, deck):
        ''' Draw two cards.'''
        self.hand += deck.draw() + deck.draw()
        
    def evaluate_hand(self):
        ''' Do we have blackjack or bust?'''

        hard_score = 0 # Aces = 11
        
        for card in self.hand:
            hard_score += cards_dict[card]
        
        soft_score = hard_score # Aces = 1
        if hard_score>21:
            n_aces = len([card for card in self.hand if card == 'A'])
            if n_aces>0:
                for ace in range(1, n_aces+1):
                    soft_score -= 10 #  -11 + 1
                    if soft_score == 21:
                        self.status = 'win'
        elif hard_score ==21:
            self.status = 'win'

        if soft_score>21: # Also catches when hard_score >21 and converting Aces to 1 doesn't save player.
            self.status = 'lose'
    
#         print(min([soft_score,hard_score])) # For QC, print total value

class Human(Player):

    def move(self, dk):
        decision = ''
        while (decision != 'h') and (decision != 's'):
            decision = input("Hit (h) or Stand (s)?")
        if decision == 'h':
            self.hand += dk.draw()
            self.print_hand()
            self.evaluate_hand()
        if decision == 's':
            self.status = 'stand'

class Dealer(Player):
    
    def move(self, dk):
        self.status = 'lose' # PLACEHOLDER. CONTINUE HERE
        #         ''' Rules to make next move''
    
    def print_hand(self, is_deal=False): 
        '''Override the regular Player print function to handle the hidden Dealer card at deal.'''
        
        if is_deal: 
            hand_formatted = str(self.hand[0]) + ' ' + '?'
        else:
            hand_formatted = ' '.join(self.hand)
        
        print('{}: {}'.format(self.name, hand_formatted))

class Table():
            
    def _new_deck(self):
        self._cards = list(cards_dict)*4

    def __init__(self):
        
        self.is_active = True
        
        # Create a new deck
        self._new_deck()

        # Add humans
        player_names = []
        while not player_names: # While is to handle empty input
            player_names = input("What's your name?").split(',') 
            
        self.players = [Human(name) for name in player_names]

        # Add dealer
        self.players += [Dealer('Encore')]
                
    def draw(self):
        ''' Takes 1 card out of the deck. Depletes deck by 1.'''
        
        n_cards =len(self._cards)
        
        # If we're out of cards, get a new deck.
        if n_cards <0: 
            self._new_deck()

        new_card = self._cards[randint(0,n_cards-1)] # Shuffles when drawing a card. Inelegant. Fix later.
        self._cards.remove(new_card)
        return([new_card])

    def play_game(self):

        for p in self.players:
            p.deal(deck)
            if isinstance(p, Dealer): # This is inelegant. Just always pass is_deal?
                p.print_hand(is_deal = True)
            else:
                p.print_hand()
            p.evaluate_hand()

        # Does dealer have a natural blackjack?
        if self.players[-1].status == 'win':
            for p in self.players[0:-1]: # For all non-dealer players 
                # Does player also have a natural blackjack?
                if p.status == 'win':
                    p.status = 'push'
                else:
                    p.status = 'lose'
                print(p.name, p.status)

        else:
            # Play out the hands of each player, one at a time.
            for p in self.players: 
                print('QC:', p.name)
                while p.status == 'open':
                    p.move(deck)
                print(p.name, p.status)
                p.status = 'open' # Reset status

In [ ]:
table = Table()

while table.is_active:
    table.play_game()
    table.is_active = input('Play again? (y)') == 'y'

What's your name? c


c: 5 2
Encore: 5 ?


Hit (h) or Stand (s)? s


c stand
Encore lose


Play again? (y) y


c: 5 2 10 Q
Encore: 5 ?
c lose
Encore lose


In [36]:
%debug

> <ipython-input-34-0e72d2d172a6>(65)move()
     63             decision = input("Hit (h) or Stand (s)?")
     64         if decision == 'h':
---> 65             self.hand += dk.draw()
     66             self.print_hand()
     67             self.evaluate_hand()



ipdb>  self.hand


['6', '5']


ipdb>  dk.draw()


['10']


...

ipdb>  


['6', '5', 'K']


ipdb>  


['6', '5', 'K']


ipdb>  exit
